In [2]:
import json
import math
import csv

In [25]:
with open('RaptorsVHornets.json') as f:
    data = json.load(f)

events = data['events']

In [26]:
events[0]["home"]["players"]

[{'lastname': 'Johnson',
  'firstname': 'James',
  'playerid': 201949,
  'jersey': '3',
  'position': 'F'},
 {'lastname': 'Scola',
  'firstname': 'Luis',
  'playerid': 2449,
  'jersey': '4',
  'position': 'F'},
 {'lastname': 'Carroll',
  'firstname': 'DeMarre',
  'playerid': 201960,
  'jersey': '5',
  'position': 'F'},
 {'lastname': 'Joseph',
  'firstname': 'Cory',
  'playerid': 202709,
  'jersey': '6',
  'position': 'G'},
 {'lastname': 'Lowry',
  'firstname': 'Kyle',
  'playerid': 200768,
  'jersey': '7',
  'position': 'G'},
 {'lastname': 'Biyombo',
  'firstname': 'Bismack',
  'playerid': 202687,
  'jersey': '8',
  'position': 'C-F'},
 {'lastname': 'DeRozan',
  'firstname': 'DeMar',
  'playerid': 201942,
  'jersey': '10',
  'position': 'G'},
 {'lastname': 'Valanciunas',
  'firstname': 'Jonas',
  'playerid': 202685,
  'jersey': '17',
  'position': 'C'},
 {'lastname': 'Caboclo',
  'firstname': 'Bruno',
  'playerid': 203998,
  'jersey': '20',
  'position': 'F'},
 {'lastname': 'Ross',
  '

In [27]:
home_players = events[0]["home"]["players"]
away_players = events[0]["visitor"]["players"]

with open('players.csv', 'w', newline='') as csvfile:
    fieldnames = ['player name', 'position', 'playerid']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for p in home_players:
        writer.writerow({
            'player name': f"{p['firstname']} {p['lastname']}",
            'position': p['position'],
            'playerid': p['playerid']
        })
    for x in away_players:
        writer.writerow({
            'player name': f"{x['firstname']} {x['lastname']}",
            'position': x['position'],
            'playerid': x['playerid']
        })

In [39]:
SHOT_HEIGHT_THRESHOLD = 3.0
BASKET_POSITIONS = [(4, 25, 10), (90, 25, 10)]  # Both baskets
SHOT_PROXIMITY_RADIUS = 3.0
MIN_TIME_BETWEEN_SHOTS = 1.0 # Minimum time in seconds between detected shots

# Standard quarter length in seconds (NBA quarters are 12 minutes)
QUARTER_LENGTH_SECONDS = 12 * 60

def distance3d(p1, p2):
    """Calculates the 3D Euclidean distance between two points."""
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2) ** 0.5

def distance2d(p1, p2):
    """Calculates the 2D Euclidean distance between two points (ignoring Z-axis)."""
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2) ** 0.5

def seconds_to_mmss(seconds):
    """Converts seconds into MM:SS.ms format."""
    minutes = int(seconds // 60)
    sec = seconds % 60
    return f"{minutes}:{sec:05.2f}"

def calculate_absolute_game_time(quarter, game_clock):
    """
    Calculates the total elapsed time in the game in seconds.
    game_clock is time remaining in the current quarter.
    """
    # Time elapsed in previous quarters
    time_from_previous_quarters = (quarter - 1) * QUARTER_LENGTH_SECONDS
    # Time elapsed in the current quarter
    time_elapsed_in_current_quarter = QUARTER_LENGTH_SECONDS - game_clock
    return time_from_previous_quarters + time_elapsed_in_current_quarter

shots = []
last_shot_absolute_time = None # This will now store the absolute game time of the last shot

# --- Start of your event processing loop (assuming 'events' is defined) ---
# Example 'events' structure for testing (replace with your actual data source):
# events = [{
#     'moments': [
#         # Quarter 1
#         [1, 0, 719.5, 0, 0, [[-1, -1, 47.5, 25.0, 1.0], [1, 201949, 40.0, 20.0, 0.0], [2, 2449, 50.0, 30.0, 0.0]]],
#         [1, 0, 710.0, 0, 0, [[-1, -1, 47.5, 25.0, 15.0], [1, 201949, 40.0, 20.0, 0.0], [2, 2449, 50.0, 30.0, 0.0]]], # Shot 1
#         [1, 0, 709.0, 0, 0, [[-1, -1, 47.5, 25.0, 1.0], [1, 201949, 40.0, 20.0, 0.0], [2, 2449, 50.0, 30.0, 0.0]]],
#         # Quarter 2
#         [2, 0, 719.0, 0, 0, [[-1, -1, 47.5, 25.0, 1.0], [1, 201949, 40.0, 20.0, 0.0], [2, 2449, 50.0, 30.0, 0.0]]],
#         [2, 0, 700.0, 0, 0, [[-1, -1, 47.5, 25.0, 12.0], [1, 201949, 40.0, 20.0, 0.0], [2, 2449, 50.0, 30.0, 0.0]]], # Shot 2
#         # Example with a box-out scenario
#         [2, 0, 690.0, 0, 0, [
#             [-1, -1, 47.5, 25.0, 1.0], # Ball
#             [1, 1001, 5.0, 25.0, 0.0],  # Boxer (Team 1, close to basket)
#             [2, 2002, 7.0, 26.0, 0.0],  # Target (Team 2, slightly further)
#             [1, 1003, 10.0, 20.0, 0.0], # Other player
#             [2, 2004, 12.0, 30.0, 0.0], # Other player
#         ]],
#         [2, 0, 685.0, 0, 0, [
#             [-1, -1, 47.5, 25.0, 15.0], # Ball at height (Shot 3)
#             [1, 1001, 5.0, 25.0, 0.0],
#             [2, 2002, 7.0, 26.0, 0.0],
#             [1, 1003, 10.0, 20.0, 0.0],
#             [2, 2004, 12.0, 30.0, 0.0],
#         ]],
#     ]
# }]
#
# If 'events' is not yet defined, you'll need to load it from your data source first.
# For instance, if 'events' comes from a JSON file:
# import json
# with open('your_events_data.json', 'r') as f:
#     events = json.load(f)


for event in events:
    current_shot_pending = False

    for moment in event.get('moments', []):
        quarter = moment[0]
        game_clock_in_quarter = moment[2] # This is the time remaining in the current quarter
        player_data = moment[5]

        # Calculate the absolute game time for the current moment
        current_absolute_time = calculate_absolute_game_time(quarter, game_clock_in_quarter)

        if not player_data or len(player_data) < 1:
            continue
        
        ball = player_data[0]
        ball_pos = (ball[2], ball[3], ball[4]) # (x, y, z)
        
        # Detect shot start: ball height crosses threshold
        if not current_shot_pending and ball_pos[2] >= SHOT_HEIGHT_THRESHOLD:
            current_shot_pending = True
        
        # If a shot is pending, check for proximity to basket and height
        if current_shot_pending:
            for i, basket_pos in enumerate(BASKET_POSITIONS):
                dist_to_basket = distance3d(ball_pos, basket_pos)
                
                # Check if ball is close to basket and at or above basket height
                if dist_to_basket <= SHOT_PROXIMITY_RADIUS and ball_pos[2] >= 10.0:
                    # Check if enough time has passed since the last detected shot
                    if (last_shot_absolute_time is None) or \
                       (current_absolute_time - last_shot_absolute_time >= MIN_TIME_BETWEEN_SHOTS):
                        
                        # --- Rebound / Box-out logic ---
                        
                        # Calculate player distances to basket
                        player_distances = []
                        for p in player_data:
                            if p[1] != -1: # Ensure it's a valid player, not ball
                                player_id = p[1]
                                player_x, player_y = p[2], p[3]
                                distance = distance2d((player_x, player_y), basket_pos[:2])
                                player_distances.append((player_id, distance))
                        
                        # Estimate offense and defense teams (based on player_data)
                        team_counts = {}
                        team_positions = {}

                        for p in player_data:
                            team_id = p[0]
                            player_id = p[1]
                            if player_id == -1: # Skip ball
                                continue
                            if team_id not in team_counts:
                                team_counts[team_id] = 0
                                team_positions[team_id] = []
                            team_counts[team_id] += 1
                            team_positions[team_id].append((p[2], p[3]))  # x, y

                        box_out_candidates = []
                        basket_xy = basket_pos[:2]

                        # Collect all valid cross-team pairs that meet the box out criteria
                        for i_p1, p1 in enumerate(player_data):
                            if p1[1] == -1: # Skip ball
                                continue
                            team1, id1, x1, y1 = p1[0], p1[1], p1[2], p1[3]
                            pos1 = (x1, y1)
                            dist1 = distance2d(pos1, basket_xy)

                            for i_p2, p2 in enumerate(player_data):
                                if p2[1] == -1 or p2[0] == team1: # Skip ball or same team
                                    continue
                                team2, id2, x2, y2 = p2[0], p2[1], p2[2], p2[3]
                                pos2 = (x2, y2)
                                dist2 = distance2d(pos2, basket_xy)
                                dist_between = distance2d(pos1, pos2)

                                # Only add if one player is clearly boxing out the other
                                # Criteria: Boxer is closer to basket, players are close, boxer is within reasonable distance
                                if dist1 < dist2 and dist_between <= 5 and dist1 <= 15:
                                    box_out_candidates.append({
                                        'BoxerID': id1,
                                        'TargetID': id2,
                                        'BoxerTeam': team1,
                                        'TargetTeam': team2,
                                        'BoxerDistanceToBasket': dist1,
                                        'TargetDistanceToBasket': dist2,
                                        'DistanceBetweenPlayers': dist_between
                                    })

                        # Resolve duplicates: only keep closest pair per player (either side)
                        box_outs = []
                        paired_ids = set()  # Keep track of players already involved in a box-out

                        # Sort all candidates by distance between players (closest first)
                        box_out_candidates.sort(key=lambda x: x['DistanceBetweenPlayers'])

                        for pair in box_out_candidates:
                            boxer = pair['BoxerID']
                            target = pair['TargetID']
                            # Add the pair only if neither player has been paired yet
                            if boxer not in paired_ids and target not in paired_ids:
                                box_outs.append(pair)
                                paired_ids.add(boxer)
                                paired_ids.add(target)

                        # --- End Rebound / Box-out logic ---

                        # Add full shot info including box outs
                        shots.append({
                            'Quarter': quarter,
                            # Store the raw numerical game_clock_in_quarter for CSV rounding
                            'GameClock': float(game_clock_in_quarter), # Ensure it's a float
                            'AbsoluteGameTime': current_absolute_time,
                            'Basket': 'Left' if i == 0 else 'Right',
                            'PlayerDistances': player_distances,
                            'BoxOuts': box_outs
                        })

                        last_shot_absolute_time = current_absolute_time # Update with absolute time
                    current_shot_pending = False # Reset shot pending state after a shot is detected
                    break  # No need to check both baskets once a shot is confirmed for one

In [40]:
shots

[{'Quarter': 1,
  'GameClock': 701.5,
  'AbsoluteGameTime': 18.5,
  'Basket': 'Left',
  'PlayerDistances': [(2449, 7.227846134727828),
   (201960, 23.47310544278494),
   (200768, 30.37632073567337),
   (201942, 36.489783655823445),
   (202685, 5.33519194403538),
   (101107, 3.3186989665831392),
   (201587, 4.971868009239584),
   (202689, 7.227846134727828),
   (203469, 6.583314754749312),
   (203798, 16.194342244388935)],
  'BoxOuts': [{'BoxerID': 201587,
    'TargetID': 202685,
    'BoxerTeam': 1610612766,
    'TargetTeam': 1610612761,
    'BoxerDistanceToBasket': 4.971868009239584,
    'TargetDistanceToBasket': 5.33519194403538,
    'DistanceBetweenPlayers': 1.332788787542872}]},
 {'Quarter': 1,
  'GameClock': 686.03,
  'AbsoluteGameTime': 33.97000000000003,
  'Basket': 'Right',
  'PlayerDistances': [(2449, 18.910678555210545),
   (201960, 4.972921642435154),
   (200768, 11.980551791541156),
   (201942, 6.097121948099774),
   (202685, 5.763541187334394),
   (101107, 33.00278095082444

In [41]:
with open('shots_with_boxouts.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'Quarter',
        'Game Clock (s)',
        'Basket',
        'Boxer ID',
        'Target ID',
        'Boxer Team',
        'Target Team',
        'Boxer Distance to Basket (ft)',
        'Target Distance to Basket (ft)',
        'Distance Between Players (ft)'
    ])

    for shot in shots:
        for boxout in shot.get('BoxOuts', []):
            writer.writerow([
                f"Q{shot['Quarter']}",
                round(shot['GameClock'], 2),
                shot['Basket'],
                boxout['BoxerID'],
                boxout['TargetID'],
                boxout['BoxerTeam'],
                boxout['TargetTeam'],
                round(boxout['BoxerDistanceToBasket'], 2),
                round(boxout['TargetDistanceToBasket'], 2),
                round(boxout['DistanceBetweenPlayers'], 2)
            ])

In [38]:
if shots: # Only write if there are any shots detected
    with open('shots_with_boxouts.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            'Quarter',
            'Game Clock (s)',
            'Basket',
            'Boxer ID',
            'Target ID',
            'Boxer Team',
            'Target Team',
            'Boxer Distance to Basket (ft)',
            'Target Distance to Basket (ft)',
            'Distance Between Players (ft)'
        ])

        for shot in shots:
            # Iterate through each boxout within the shot
            for boxout in shot.get('BoxOuts', []):
                writer.writerow([
                    f"Q{shot['Quarter']}", # Format Quarter as 'Q1', 'Q2', etc.
                    round(shot['GameClock'], 2), # Round the numerical game clock
                    shot['Basket'],
                    boxout['BoxerID'],
                    boxout['TargetID'],
                    boxout['BoxerTeam'],
                    boxout['TargetTeam'],
                    round(boxout['BoxerDistanceToBasket'], 2),
                    round(boxout['TargetDistanceToBasket'], 2),
                    round(boxout['DistanceBetweenPlayers'], 2)
                ])
    print("Box-out data successfully written to shots_with_boxouts.csv")
else:
    print("No shots with box-outs detected to write to CSV.")

TypeError: type str doesn't define __round__ method